# COVID-19 Epidemiology Modelling Tutorial

---

## How to use this tutorial:

At no point will you need to actually write code - you can interact with this tutorial by playing around with parameters in the already-written code. The one thing you have to be able to do is run the code in this notebook. The 'notebook' is the page you are on - it is made up of cells of text (like what you are reading now), and cells of code (grey regions with coloured fonts or 'Show code' written in blue if the code is hidden). To run the code in a cell, you simply click on the 'play' icon that shows up in the top left when you hover your cursor over the cell. Try running the cell below:

In [ ]:
print('Well done, you just ran your first cell of Python code!')

As you can see after running the cell, an output is printed below the code. The output of the code above is just the message that was specified to be printed, but we can do more useful things in the code and output the results.

The code below multiplies one number by another and outputs the result. Specify two numbers you want to multiply by replacing the '2' and '3' and then run the cell to print the result:

In [ ]:
number_1 = 2 # Insert any number here
number_2 = 3 # Insert any number here
product = number_1 * number_2
print(f"{number_1} times {number_2} is {product}")

## Overview

This tutorial focuses on modelling the spread of disease during epidemics using Physics-Informed Neural Networks (PINNs). Modelling in epidemiology allows for oncoming spikes and dips in cases of a disease to be predicted. This information can be used to guide public health policy decisions.

We will start by applying a neural networks to predict the motion of a damped harmonic oscillator (a spring or pendulum with friction or air resistance being acknowledged), following the procedure described [here](https://benmoseley.blog/my-research/so-what-is-a-physics-informed-neural-network/) to learn to apply the PINN framework. Then, we will apply the same method to COVID-19 data from the US in 2020 to try to predict the spikes and dips in COVID cases that occurred.

To start, run the following cell to import the necessary libraries:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import clear_output

In order to run a neural network, we need a dataset to train and make predictions on. We will use the analytic solution of the damped harmonic oscillator (essentially just the motion of a spring) for this purpose, training our neural network on the initial motion of the oscillator then testing it by asking it to predict the trajectory after the training period.

The following cell sets up the required data and plots the trajectory along with the points along the trajectory that the neural network will be trained on:

In [ ]:
# @title
def oscillator(d, w0, t):
    assert d < w0
    w = np.sqrt(w0**2-d**2)
    phi = np.arctan(-d/w)
    A = 1/(2*np.cos(phi))
    cos = torch.cos(phi+w*t)
    sin = torch.sin(phi+w*t)
    exp = torch.exp(-d*t)
    y  = exp*2*A*cos
    return y

d, w0 = 2, 20

# get the analytical solution over the full domain
t = torch.linspace(0,1,500).view(-1,1)
x = oscillator(d, w0, t).view(-1,1)

# slice out a small number of points from the LHS of the domain
t_data = t[0:300:25]
x_data = x[0:300:25]

plt.figure()
plt.plot(t, x, label="Exact solution")
plt.xlabel('time t')
plt.ylabel('position x')
plt.scatter(t_data, x_data, color="tab:orange", label="Training data")
plt.legend()
plt.show()

Neural networks are essentially a very flexible type of function. For any input values given to them, they output a specific output value or set of values. In our case there is one input and one output - we pass a time in and get the position of the oscillator out. To get a trajectory we ask it for outputs from a set of lots of different input times, to get the position of the oscillator at each time.

At first when we create the neural network, asking it for the position of the oscillator at a given time is pointless - it doesn't know anything about the oscillator. However, because neural networks are very flexible and we know the correct answer, we can update the neural network based on how wrong it is at the points in time that we ask it about. We call the measure of how wrong it is the 'loss'. Repeatedly updating the neural network to decrease the loss 'trains' the neural network to know the positions it should output for different input times.

Our aim is to train a neural network using the orange points, in hopes that it will output the entire blue curve as the trajectory after it has been trained. To do this, we will create a simple 'feed forward' neural network with one input (a time value) and one output (a position value) using PyTorch in the following cell:

In [ ]:
# @title
class ff_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1,32)
        self.fc2 = nn.Linear(32,32)
        self.fc3 = nn.Linear(32,32)
        self.fc4 = nn.Linear(32,32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

Now, we will train the model on the first portion of the analytic trajectory in this cell. The outputted plots are the trajectory that the neural network predicts for the oscillators at different stages of the training process.

In [ ]:
# @title
torch.manual_seed(123) # This is to ensure the same random numbers are used each run, so the results are reproducible
model = ff_NN()
optimiser = torch.optim.Adam(model.parameters(),lr=1e-3)

for i in range(2500):
    optimiser.zero_grad()
    x_nn = model(t_data)
    loss = torch.mean((x_nn-x_data)**2)
    loss.backward()
    optimiser.step()

    # plot the result as training progresses
    if (i+1) % 100 == 0 or i == 0:
        clear_output(wait=True)
        x_result = model(t).detach()
        plt.figure(figsize=(8,4))
        plt.plot(t,x, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
        plt.plot(t,x_result, color="tab:blue", linewidth=2, alpha=0.8, label="Neural network prediction")
        plt.scatter(t_data, x_data, s=60, color="tab:orange", alpha=0.4, label='Training data')
        l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
        plt.setp(l.get_texts(), color="k")
        plt.xlim(-0.05, 1.05)
        plt.ylim(-1.1, 1.1)
        plt.text(1.065,0.7,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
        plt.axis("off")
        plt.show()

Clearly, the neural network correctly learned to interpolate the trajectory of the oscillator between the training points. However, the neural network's prediction for the trajectory of the oscillator after this period is incorrect and physically nonsensical - why would a spring wobble around for a while then suddenly start stretching out of its own accord?


## Introducing PINNs
This is where PINNs come in. If we know some equations which physical reasoning dictates the system should obey, we can include the extent to which these equations are violated in the loss function used during training. Then, the neural network's output should not only be able to interpolate the trajectory in the period on which it was trained, but it should be able to make a physically-motivated guess about the subsequent motion by combining interpolation of training datapoints with the 'physical intuition' which we have granted it.

For a damped harmonic oscillator, the equation we can impose of this type is a second order differential equation which is simply $F=ma$ for this system:

\begin{equation}
m\frac{d^2x}{{dt}^2} +\mu \frac{dx}{dt}+kx=0
\end{equation}

$\mu$ and $k$ are constants that define the motion of the system, but we can learn their values as well as the correct weights of the neural network during training on the initial motion of the oscillator. Run the following cell to see this in action (this will take a bit longer than last time):

In [ ]:
# @title
# Assemble a set of points across the entire period of time that we want to solve the trajectory for
t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)

# Initialise the model and parameters
torch.manual_seed(123) # This is to ensure the same random numbers are used each run, so the results are reproducible
PINN_model = ff_NN()
mu = torch.tensor([5.0], requires_grad=True)
k = torch.tensor([360.0], requires_grad=True)
PINN_optimiser = torch.optim.Adam(PINN_model.parameters(),lr=5e-4)
params_optimiser = torch.optim.Adam([mu,k],lr=3e-3)

# Run training loop
for i in range(30000):
    PINN_optimiser.zero_grad()
    params_optimiser.zero_grad()

    # compute the "data loss"
    x_nn = PINN_model(t_data)
    loss1 = torch.mean((x_nn-x_data)**2)# use mean squared error

    # compute the "physics loss"
    x_nn_physics = PINN_model(t_physics)
    dx  = torch.autograd.grad(x_nn_physics, t_physics, torch.ones_like(x_nn_physics), create_graph=True)[0]# computes dy/dx
    dx2 = torch.autograd.grad(dx,  t_physics, torch.ones_like(dx),  create_graph=True)[0]# computes d^2y/dx^2
    physics = dx2 + mu*dx + k*x_nn_physics # computes the residual of the 1D harmonic oscillator differential equation
    loss2 = (1e-4)*torch.mean(physics**2)

    # backpropagate joint loss
    loss = loss1 + loss2# add two loss terms together
    loss.backward()
    PINN_optimiser.step()
    # only start optimising mu and k after the interpolation has begun
    if i > 5000:
        params_optimiser.step()


    # plot the result as training progresses
    if (i+1) % 500 == 0 or i == 0:
        clear_output(wait=True)
        x_results = PINN_model(t).detach()
        plt.figure(figsize=(8,4))
        plt.plot(t.detach().numpy(),x, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
        plt.plot(t.detach().numpy(),x_results, color="tab:blue", linewidth=2, alpha=0.8, label="Neural network prediction")
        plt.scatter(t_data.detach().numpy(), x_data, s=60, color="tab:orange", alpha=0.4, label='Training data')
        l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
        plt.setp(l.get_texts(), color="k")
        plt.xlim(-0.05, 1.05)
        plt.ylim(-1.1, 1.1)
        plt.text(1.065,0.7,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
        plt.axis("off")
        plt.show()
        print(f'mu={mu.item()}, k={k.item()}')

Clearly, the PINN framework is much better at predicting the trajectory of the oscillator. Further, the correct values of the parameters in the model were successfully learned to within about a 1% error!

All of this is promising - we want to make predictions about the dynamics of an epidemic and we have just seen that PINNs can combine information from past data with models for how we expect the system to react to get good predictions.

## Predicting COVID-19 Trends

Before we jump in applying the PINN framework to the COVID-19 prediction problem, let's first see how well a simple neural network trained to fit the data does in predicting future trends.

The following cell imports US national statistics on the COVID-19 pandemic from a [New York Times dataset](https://github.com/nytimes/covid-19-data/blob/master/us.csv):

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv'
# Load the CSV file into a DataFrame
df = pd.read_csv(url)

# Calculate the 'Currently Infected' count
df['Currently Infected'] = df['cases'] - df['deaths']
# Calculate the rolling difference over the previous ten days
df['Infected'] = df['Currently Infected'].diff(periods=10)
# Calculate recovered count:
df['Recovered'] = df['Currently Infected'].diff(periods=10)

x = np.array(df['Infected'][60:300]).astype(np.float32)/1e6
t = np.arange(len(df))[60:300]
t = (t/239).astype(np.float32)

x = torch.from_numpy(x).view(-1,1)
t = torch.from_numpy(t).view(-1,1)

# slice out a small number of points from the LHS of the domain
t_data = t[0:110:5]
x_data = x[0:110:5]

plt.figure()
plt.plot(t, x, label="Exact solution")
plt.scatter(t_data, x_data, color="tab:orange", label="Training data")
plt.ylabel('Number of people with COVID (Millions)')
plt.legend()
plt.show()

Again, our aim will be to train a simple feed-forward neural network with tanh as the activation function on the orange datapoints. Then, we will ask the neural network for a prediction across the entire range of time plotted above.

In [ ]:
# @title
class COVID_NN(nn.Module):
    # Put your code here:
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,64)
        self.fc5 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

torch.manual_seed(123) # This is to ensure the same random numbers are used each run, so the results are reproducible
COVID_model = COVID_NN() # Initialise the model
COVID_optimiser = torch.optim.Adam(COVID_model.parameters(),lr=1e-2) # Define the optimiser

# Training loop
for i in range(5000):
    COVID_optimiser.zero_grad() # Zero the gradient stored in the optimiser
    x_nn = COVID_model(t_data) # Call the neural network on t_data
    loss = torch.mean((x_nn-x_data)**2) # Use the mean squared error as the loss
    loss.backward() # Backpropogate the loss
    COVID_optimiser.step() # Take a step with the optimiser

    # plot the result as training progresses
    if (i+1) % 1000 == 0 or i == 0:
        clear_output(wait=True)
        x_results = COVID_model(t).detach()
        plt.figure(figsize=(6,4))
        plt.plot(t.detach().numpy(),x, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
        plt.plot(t.detach().numpy(),x_results, color="tab:blue", linewidth=2, alpha=0.8, label="Neural network prediction")
        plt.scatter(t_data.detach().numpy(), x_data, s=60, color="tab:orange", alpha=0.4, label='Training data')
        l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
        plt.setp(l.get_texts(), color="k")
        plt.xlim(0.2, 1.3)
        plt.ylim(0, 1.4)
        plt.text(1.3,1.0,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
        plt.axis("off")
        plt.show()

Just like in the case of the oscillator, this naive approch is successful in interpolating the behaviour between the training datapoints, but is not of any real value for making predictions about the future.

To apply a PINN, we need an analogue of the $F=ma$ differential equation for this system. There are plenty of models for epidemiology that can supply us with differential equations to use for this purpose - for this reason, several academic papers applying different epidemiological models in this way have been written. We will use the simplest model, which is known as the SIR model.

SIR stands for Susceptible, Infected, Removed - those are the three classes that we split the population up into in this model. The susceptible group is the set of all people who haven't had the virus yet, the infected group is the set of people who are currently infected by the virus so are transmitters of it, and the removed group is the set of all people who can no longer be infected or transmit the virus (i.e. the set of people who recovered from or died due to the virus). We define the number of people in each group as $S(t)$, $I(t)$, and $R(t)$. Assuming a very large population (a valid assumption for our dataset), the population is approximately constant, so $S(t)+I(t)+R(t)=N$.

Taking $\gamma$ as the rate of recovery or death - about 0.1 days$^{-1}$ in the case of COVID-19, and $\beta$ as the transmission rate, the movement of people from one group to another can be described by the following differential equations:

$$\frac{dS}{dt}=-\frac{\beta S I}{N}$$
$$\frac{dI}{dt}=\frac{\beta S I}{N} - \gamma I$$
$$\frac{dR}{dt}= \gamma I$$

This simply says that rate of change changes in the number of infected people is proportional to the number of infected people times the fraction of the population that is susceptible minus the rate of recovery. We are only learning and predicting $I(t)$, so we would like to capture this information in a differential equation in terms of $I(t)$ only. We can do this by taking the first and last of these differential equations as definitions of the system.

Using $S(t)+I(t)+R(t)=N$ and $\frac{dR}{dt}= \gamma I$, we get
$$S(t) = N- I(t)-\gamma\int\limits_{0}^{t}I(t')dt'$$
Subbing this into the second differential equation, we get a condition for the system to obey if it satisfies the SIR model:
$$ \frac{\beta I(t)}{N}\left(N-I(t)-\gamma\int\limits_{0}^{t}I(t')dt'\right) - \gamma I(t) - \frac{dI}{dt}=0$$

Using the PINN framework, the left hand side of this expression will be included in our loss function. Run the following cell to apply this and see the predicted trajectory at different stages of training change:

In [ ]:
# @title
def get_R(I):
    # Start coding here:
    R = torch.zeros_like(I,requires_grad=True)
    end = np.floor(len(R)/2).astype(int)
    for count in range(1,end):
        ones = torch.zeros_like(I)
        ones[:len(R)-2*count] = 1.0
        R = R + torch.roll(I*ones, 2*count)
    return R

t_physics = t[::5].requires_grad_(True) # These are the points in time where we will evaluate the physics loss
g = 23.9 # This is the value of gamma that you get by rescaling 1/(10 days) with 239 days = 1 unit

torch.manual_seed(111) # This is to ensure the same random numbers are used each run, so the results are reproducible
PINN_COVID_model = COVID_NN()
optimiser = torch.optim.Adam(PINN_COVID_model.parameters(),lr=1e-4)
beta = torch.ones_like(t_physics, requires_grad=True)
beta_optimiser = torch.optim.Adam([beta],lr=5e-4)

# Training loop:
for i in range(9500):
    optimiser.zero_grad() # Zero the gradient stored in the optimiser

    # compute the "data loss"
    x_nn = PINN_COVID_model(t_data) # Pass t_data to the model
    loss1 = torch.mean((x_nn-x_data)**2) # Use mean squared error

    # compute the "physics loss"
    x_nn_physics = PINN_COVID_model(t_physics) # Pass t_physics to the model
    dx  = torch.autograd.grad(x_nn_physics, t_physics, torch.ones_like(x_nn_physics), create_graph=True)[0] # Compute dy/dx
    physics = dx + g*x_nn_physics - (13*beta*x_nn_physics/330)*(330 - x_nn_physics - get_R(x_nn_physics)) # Compute the residual of the equation
    loss2 = (1e-4)*torch.mean(physics**2)

    # backpropagate the joint loss - no need to wait for a certain number of steps before starting to optimise beta
    loss = loss1 + loss2 # add the two loss terms together
    loss.backward() # Backpropogate the loss
    optimiser.step() # Take a step with the neural network optimiser
    beta_optimiser.step() # Take a step with the beta optimiser

    # plot the result as training progresses
    if (i+1) % 500 == 0:
        clear_output(wait=True)
        x_results = PINN_COVID_model(t).detach()
        plt.figure(figsize=(6,4))
        plt.plot(t.detach().numpy(),x, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
        plt.plot(t.detach().numpy(),x_results, color="tab:blue", linewidth=2, alpha=0.8, label="Neural network prediction")
        plt.scatter(t_data.detach().numpy(), x_data, s=60, color="tab:orange", alpha=0.4, label='Training data')
        l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
        plt.setp(l.get_texts(), color="k")
        plt.xlim(0.2, 1.3)
        plt.ylim(0, 1.4)
        plt.text(1.3,1,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
        plt.axis("off")
        plt.show()

Although the PINN clearly didn't manage to capture the spike at the end of the time period we're looking at, it was much more successful than the 'physics-less' neural network in predicting that there would be a peak and subsequent fall-off after the end of the training period.

We trained the model for long enough for it to fit the data and had we ran it for longer, it's possible that the output would change and no longer be quite so correct. This may seem like we're cheating and getting the correct answer because we know what it is. In a sense it's true that knowing the correct prediction is very helpful in knowing when to stop training, but this is a general flaw of using neural networks for this type of predictive task. The question of when to stop training when working with neural networks is more of an art than an exact science; train too long and you will 'overfit' the data (i.e. fit the training data really well, but not generalise well beyond the training set), but train too little and you risk losing important information hidden in the training set.

The problem would be much simpler to solve if we weren't constricted to data on just one COVID-19 epidemic. If we had data on thousands of different epidemics, we could abandon the PINN idea and completely change the architecture of the neural network, so that its job is to predict the cases during the second half of the time period based on knowledge of the first half of the period. In this case, overfitting would be much easier to monitor without ever checking the output of the model for the epidemic that we want to find out information about.

Using more detailed models of the epidemic dynamics such as the SAIRD model (which differentiates between symptomatic and assymptomatic, recovered and dead) could help make more accurate, longer term predictions with our PINN - although we did adjust the SIR model by giving $\beta$ the ability to vary with time, it is still quite a crude model of a rather complex system. With that said, there is a chance it could be improved by simply making the time dependence of $\beta$ more informed by including information on current affairs such as changes in public health policy in the dataset (using newspaper headlines perhaps).

As mentioned earlier, there are several academic papers and preprints out there on the topic of using PINNs to retrospectively predict COVID-19 spikes and dips - here are some links if you feel like having a read:

[Approaching epidemiological dynamics of COVID-19 with physics-informed neural networks](https://arxiv.org/abs/2302.08796)

PINN Training using Biobjective Optimization:
The Trade-off between Data Loss and Residual Loss - [Github](https://git.uni-wuppertal.de/heldmann/covid-prediction-pinn) & [ArXiv](https://arxiv.org/abs/2302.01810)